In [19]:
import pandas as pd
from tabulate import tabulate  # tabulate 라이브러리 활용

class IndustryDataAnalyzer:
    def __init__(self, file_paths):
        """생성자: 파일 경로를 받아 데이터 로드"""
        self.file_paths = file_paths
        self.data_frames = {}
        self.all_companies = {}
        self.load_data()

    def load_data(self):
        """엑셀 파일 데이터를 로드"""
        for category, path in self.file_paths.items():
            try:
                df = pd.read_excel(path, engine="openpyxl")
                self.data_frames[category] = df
                # 모든 기업 이름을 저장 (중복 제거)
                if not df.empty:
                    self.all_companies.update({company: category for company in df.iloc[:, 0].dropna()})
            except Exception as e:
                print(f"{category} 파일을 불러오는 중 오류 발생: {e}")

    def display_all_companies(self):
        """모든 기업 이름을 산업군별로 보기 좋게 나열"""
        print("[전체 기업 목록]")
        industries = {}
        for company, category in self.all_companies.items():
            if category not in industries:
                industries[category] = []
            industries[category].append(company)

        for category, companies in industries.items():
            print(f"[{category}]")
            for company in companies:
                print(f"  - {company}")

    def display_company_data(self, company_name):
        """특정 기업의 데이터를 출력"""
        if company_name in self.all_companies:
            category = self.all_companies[company_name]
            df = self.data_frames[category]
            company_data = df[df.iloc[:, 0] == company_name]
            print(f"\n[{category}] {company_name} 데이터")
            print(tabulate(company_data, headers='keys', tablefmt='grid'))  # 데이터 출력
        else:
            print("입력한 기업 이름이 데이터에 없습니다. 다시 확인해주세요.")

    def display_category_data(self, category):
        """선택한 카테고리 데이터를 보기 좋게 표시"""
        if category in self.data_frames:
            df = self.data_frames[category]
            print(f"[{category}] 데이터")
            print(tabulate(df, headers='keys', tablefmt='grid'))  # tabulate로 출력
        else:
            print("잘못된 입력입니다. 다시 시도해주세요.")

    def compare_companies(self, category):
        """선택한 산업군의 두 회사를 비교하고 몇 대 몇인지 표시"""
        if category in self.data_frames:
            df = self.data_frames[category]
            print("기업 목록:")
            companies = df.iloc[:, 0].dropna().tolist()
            for company in companies:
                print(f"  - {company}")

            # 기업 두 개를 한 번에 입력받도록 수정
            while True:
                company_names = input("비교하고 싶은 2개의 기업을 입력해주세요: ").strip().split()

                if len(company_names) == 2:
                    company1, company2 = company_names
                    if company1 in companies and company2 in companies:
                        c1_data = df[df.iloc[:, 0] == company1].iloc[0]
                        c2_data = df[df.iloc[:, 0] == company2].iloc[0]
                        print(f"\n[{category}] {company1}과 {company2} 비교")

                        c1_count, c2_count = 0, 0  # 두 기업의 우위를 비교하기 위한 카운터
                        for col in df.columns[1:]:
                            c1_value = c1_data[col]
                            c2_value = c2_data[col]
                            if c1_value > c2_value:
                                comparison = f"{company1}({c1_value}) > {company2}({c2_value})"
                                c1_count += 1
                            elif c1_value < c2_value:
                                comparison = f"{company1}({c1_value}) < {company2}({c2_value})"
                                c2_count += 1
                            else:
                                comparison = f"{company1}({c1_value}) = {company2}({c2_value})"
                            print(f"- {col}: {comparison}")

                        # 결과 출력
                        print(f"\n[{category}] 결과: {company1} {c1_count} 대 {company2} {c2_count}")
                        break  # 비교가 끝나면 루프 종료
                    else:
                        print("기업 이름 중 하나 이상이 데이터에 없습니다. 다시 확인해주세요.")
                else:
                    print("두 개의 기업 이름을 정확하게 입력해주세요.")
        else:
            print("잘못된 입력입니다. 다시 시도해주세요.")

    def run(self):
        """메인 프로그램 실행"""
        print("산업 데이터 분석 프로그램에 오신 것을 환영합니다!")
        while True:
            while True:
                print("\n메뉴:")
                print("1. 전체 기업 목록 및 기업 데이터 검색")
                print("2. 산업군 선택 및 기업 데이터 보기")
                print("3. 특정 기업 비교")
                print("4. 프로그램 종료")
                choice = input("메뉴에 있는 목록 중 선택하고 싶은 목록의 번호를 입력하세요: ").strip()

                if choice == "1":
                    self.display_all_companies()
                    while True:
                        company_name = input("데이터를 확인할 기업 이름을 하나 입력하세요: ").strip()
                        if company_name in self.all_companies:
                            self.display_company_data(company_name)
                            break  # 올바른 기업 입력 후, 다시 메뉴로 돌아가도록
                        else:
                            print("입력한 기업 이름이 데이터에 없습니다. 다시 입력해주세요.")
                    break  # 1번 작업을 완료한 후, 메뉴로 돌아가도록

                elif choice == "2":
                    print("산업군 목록: IT 및 소프트웨어, 제조업, 에너지 및 환경, 금융 및 컨설팅, 물류 및 유통")
                    while True:
                        category = input("산업군 목록 중 하나를 입력하세요: ").strip()
                        if category in self.data_frames:
                            self.display_category_data(category)
                            break  # 카테고리 작업을 마친 후, 메뉴로 돌아가도록
                        else:
                            print("잘못된 산업군 입력입니다. 다시 입력해주세요.")

                elif choice == "3":
                    print("산업군 목록: IT 및 소프트웨어, 제조업, 에너지 및 환경, 금융 및 컨설팅, 물류 및 유통")
                    while True:
                        category = input("비교할 기업의 산업군을 하나 입력하세요: ").strip()
                        if category in self.data_frames:
                            self.compare_companies(category)
                            break  # 비교가 끝나면 메뉴로 돌아가도록
                        else:
                            print("잘못된 산업군 입력입니다. 다시 시도해주세요.")

                elif choice == "4":
                    print("프로그램을 종료합니다. 이용해 주셔서 감사합니다!")
                    return  # 프로그램 종료

                else:
                    print("잘못된 입력입니다. 다시 시도해주세요.")

# 파일 경로 설정
file_paths = {
    "IT 및 소프트웨어": "IT 및 소프트웨어.xlsx",
    "제조업": "제조업.xlsx",
    "에너지 및 환경": "에너지 및 환경.xlsx",
    "금융 및 컨설팅": "금융 및 컨설팅.xlsx",
    "물류 및 유통": "물류 및 유통.xlsx",
}

# 클래스 인스턴스 생성 및 실행
analyzer = IndustryDataAnalyzer(file_paths)
analyzer.run()



산업 데이터 분석 프로그램에 오신 것을 환영합니다!

메뉴:
1. 전체 기업 목록 및 기업 데이터 검색
2. 산업군 선택 및 기업 데이터 보기
3. 특정 기업 비교
4. 프로그램 종료
메뉴에 있는 목록 중 선택하고 싶은 목록의 번호를 입력하세요: 2
산업군 목록: IT 및 소프트웨어, 제조업, 에너지 및 환경, 금융 및 컨설팅, 물류 및 유통
산업군 목록 중 하나를 입력하세요: 제조업
[제조업] 데이터
+----+------------------------+--------------------+------------+--------------+------------+--------------+
|    | 기업 이름              | 올해 입사자 초봉   | 복리후생   | 고용안정성   | 조직문화   | 커리어성장   |
+====+========================+====================+============+==============+============+==============+
|  0 | 한국타이어앤테크놀로지 | 3,953만 원         | 51점       | 65점         | 40점       | 78점         |
+----+------------------------+--------------------+------------+--------------+------------+--------------+
|  1 | 한온시스템             | 3,920만 원         | 42점       | 58점         | 42점       | 58점         |
+----+------------------------+--------------------+------------+--------------+------------+--------------+
|  2 | LIG넥스원              | 4,071만 원         | 53점       | 63점    

KeyboardInterrupt: Interrupted by user